In [ ]:
pathloc = str(os.getcwd()) + '/'

In [10]:
import sys 
pathloc = '/content/drive/MyDrive/Colab Notebooks/dev/aa-matching/'
if pathloc not in sys.path:
    sys.path.append(pathloc)
!pip install biopython

In [75]:
#!/usr/bin/env python3

import math
import os
import os.path
from os import wait
import pandas as pd
import numpy as np
import aa_matching_msf as aa
from pathlib import Path
from collections import defaultdict
from functools import reduce
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Data.CodonTable import TranslationError
from tqdm.notebook import trange, tqdm

In [13]:
aa_mm = aa.AAMatch(dbversion=3400)

MSF files already downloaded
MSF files already downloaded
MSF files already downloaded
MSF files already downloaded
MSF files already downloaded
MSF files already downloaded
MSF files already downloaded
MSF files already downloaded
MSF files already downloaded


In [14]:
#!/usr/bin/env python

###############################################################################
#   SCRIPT NAME:    nuc_matching_msf.py
#   DESCRIPTION:    Module for nucleotide matching functions
#   OUTPUT:
#   DATE:           September 01, 2020
#	LAST UPDATE:	January 11, 2021
#	REASON:			Auto-alignment to coordinate system.
#   AUTHOR:         Giovanni Biagini (dbiagini@tulane.edu ; GitHub: gbiagini)
#   PI:             Loren Gragert, Ph.D.
#   ORGANIZATION:   Tulane University School of Medicine
#   NOTES:			FIXME: Need to reduce redundancy through imports.
###############################################################################

import re
import os.path
from os import path
import Bio
from Bio import SeqIO
from Bio import AlignIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import pandas as pd
import requests


class NucMatch():
    # TODO (gbiagini) - wrap outer code in a main() function to avoid running
    #  every time scripts are imported

    def __init__(self, dbversion=3420, ungap=False):
        # if ungap = True, use complete reference sequence
        # if ungap = False, use gapped reference sequence
        self.dbversion = dbversion
        self.ungap = ungap
        hlaProteinOffset = {
            "A" : 0,
            "B" : 0,
            "C" : 0, 
            "DRA" : 0,
            "DRB1" : 0,
            "DRB3" : 0,
            "DRB4" : 0,
            "DRB5" : 0,
            "DQA1" : 0,
            "DQB1" : 0,
            "DPA1" : 0,
            "DPB1" : 0,
        }

        first_ten = {
            "A*01:01:01:01": "GGCTCCCACTCCATGAGGTATTTCTTCACA",
            "B*07:02:01:01": "GGCTCCCACTCCATGAGGTATTTCTACACC",
            "C*01:02:01:01": "TGCTCCCACTCCATGAAGTATTTCTTCACA",
            "DRB1*01:01:01:01": "GGGGACACCCGACCACGTTTCTTGTGGCAG",
            "DRB3*01:01:02:01": "GGGGACACCCGACCACGTTTCTTGGAGCTG",
            "DRB4*01:01:01:01": "GGGGACACCCAACCACGTTTCTTGGAGCAG",
            "DRB5*01:01:01:01": "GGGGACACCCGACCACGTTTCTTGCAGCAG",
            "DQA1*01:01:01:01": "GAAGACATTGTGGCTGACCACGTTGCCTCT",
            "DQB1*05:01:01:01": "AGAGACTCTCCCGAGGATTTCGTGTACCAG",
            "DPA1*01:03:01:01": "ATCAAGGCGGACCATGTGTCAACTTATGCC",
            "DPB1*01:01:01:01": "AGGGCCACTCCAGAGAATTACGTGTACCAG"
        }
        refseq_full = {
           "A" : "A*01:01:01:01",
            "B" : "B*07:02:01:01",
            "C" : "C*01:02:01:01",
            "DRB1" : "DRB1*01:01:01:01",
            "DRB3" : "DRB3*01:01:02:01",
            "DRB4" : "DRB4*01:01:01:01",
            "DRB5" : "DRB5*01:01:01:01",
            "DQA1" : "DQA1*01:01:01:01",
            "DQB1" : "DQB1*05:01:01:01",
            "DPA1" : "DPA1*01:03:01:01",
            "DPB1" : "DPB1*01:01:01:01",
        }

        refseq = {
            "A" : "A*01:01",
            "B" : "B*07:02",
            "C" : "C*01:02",
            "DRB1" : "DRB1*01:01",
            "DRB3" : "DRB3*01:01",
            "DRB4" : "DRB4*01:01",
            "DRB5" : "DRB5*01:01",
            "DQA1" : "DQA1*01:01",
            "DQB1" : "DQB1*05:01",
            "DPA1" : "DPA1*01:03",
            "DPB1" : "DPB1*01:01",
            }

            # use only ARD positions
        ard_start_pos = {
            "A" : 1,
            "B" : 1,
            "C" : 1,
            "DRB1" : 1,
            "DRB3" : 1,
            "DRB4" : 1,
            "DRB5" : 1,	
            "DQA1" : 1,
            "DQB1" : 1,
            "DPA1" : 1,
            "DPB1" : 1,
        }
        ard_end_pos = {
            "A" : 182,
            "B" : 182,
            "C" : 182,
            "DRB1" : 94,
            "DRB3" : 94,
            "DRB4" : 94,
            "DRB5" : 94,	
            "DQA1" : 94,
            "DQB1" : 95, #increased by 1
            "DPA1" : 94,
            "DPB1" : 94,
        }

        # lots of incomplete ARD sequences in IMGT/HLA
        # this should be handled in reference alignment, not here
        ard_start_pos_incomplete = {
            "A" : 2, # A*02:50
            "B" : 2, # B*07:30
            "C" : 2, # C*01:10
            "DRB1" : 7, # DRB1*08:19
            "DRB3" : 2, # TBD
            "DRB4" : 2, # TBD
            "DRB5" : 2, # TBD
            "DQA1" : 6, # DQA1*01:06
            "DQB1" : 6, # DQB1*05:100
            "DPA1" : 11, # DPA1*01:03:02
            "DPB1" : 6, # DPB1*01:01:03
        }
        ard_end_pos_incomplete = {
            "A" : 182, # A*02:50
            "B" : 182, # B*07:30
            "C" : 182, # C*01:10
            "DRB1" : 92, # DRB1*08:19
            "DRB3" : 2, # TBD
            "DRB4" : 2, # TBD
            "DRB5" : 2, # TBD
            "DQA1" : 87, # DQA1*01:06
            "DQB1" : 94, # DQB1*05:100
            "DPA1" : 84, # DPA1*01:03:02
            "DPB1" : 92, # DPB1*01:01:03
        }

        self.refseq_full = refseq_full
        self.refseq = refseq
        self.ard_start_pos = ard_start_pos
        self.ard_end_pos = ard_end_pos
        self.ard_start_pos_incomplete = ard_start_pos_incomplete
        self.ard_end_pos_incomplete = ard_end_pos_incomplete
        self.hlaProteinOffset = hlaProteinOffset
        self.first_ten = first_ten
        self.main()

        if self.ungap == True:
            self.remove_gap()

    # generate a regular expression to find first ten residues of each locus
    # ignoring '-' characters.
    def regex_gen(self):
        regexes = {}
        for each in self.first_ten.keys():
            regex = ""
            for i in range(0, len(self.first_ten[each])):
                regex += str(self.first_ten[each][i])+"[^"+self.first_ten[each]+"]*?"
            regexes[each] = regex
        return regexes

    # Align the coordinate system as appropriate to the beginning of the mature
    # protein sequence.
    def coordinate(self, regex, sequence):
        o = re.search(regex, str(sequence))
        offset = o.start()
        return offset

    def getMatureProteinOffset(self, locus):
            return self.hlaProteinOffset.get(locus, "Invalid HLA Locus")
        
    def adjust_end(self, multipleseq, loc, ard_start_pos, ard_start_pos_incomplete, ard_end_pos,
                    ard_end_pos_incomplete, prev=0, prev_inc=0):
        loc_full_allele = self.refseq_full[loc]
        full_protein = multipleseq[id==loc_full_allele].seq
        mature_protein = full_protein[self.getMatureProteinOffset(loc):]
        start = ard_start_pos
        end = ard_end_pos
        start_inc = ard_start_pos_incomplete
        end_inc = ard_end_pos_incomplete
        count = mature_protein[start:end].count('-')
        count_inc = mature_protein[start_inc:end_inc].count('-')

        check = count - prev
        check_inc = count_inc - prev_inc

        if (check == 0): 
            new_end = end + (count-prev)
            new_end_inc = end_inc + (count_inc-prev_inc)
            newlist = [new_end, new_end_inc]
            return newlist
        else:
            new_end = end + (count - prev)
            prev = count
            if (check_inc != 0):
                new_end_inc = end_inc + (count_inc - prev_inc)
                prev_inc = count_inc
                return self.adjust_end(multipleseq, loc, start, start_inc, new_end,
                                    new_end_inc, prev, prev_inc)
            else:
                new_end_inc = end_inc
                return self.adjust_end(multipleseq, loc, start, start_inc, new_end,
                                    new_end_inc, prev, prev_inc=0)

    def remove_ins(self, loc_full_alseq):
        # need to remove the inserts from the reference sequence to print into
        # the IMGT/HLA .txt file
        gapframe = pd.DataFrame.from_dict(loc_full_alseq, orient="index")
        droplist = []
        for i, row in gapframe.iterrows():
            if i == self.refseq[loc]:
                for name, data in gapframe.iteritems():
                    if data[i] == '-':
                        droplist.append(name)
                    else:
                        continue
            else:
                continue
        ungapframe = gapframe.drop(droplist, axis=1)
        for j, jrow in ungapframe.iterrows():
            jrow = jrow.to_string(header=False, index=False)
            jrow = jrow.replace('\n', '')
            jrow = jrow.replace(' ', '')
            loc_full_alseq[j] = jrow
        
        return loc_full_alseq

    def reference(self, multipleseq, loc):
        loc_full_allele = self.refseq_full[loc]
        regexes = self.regex_gen()
        mslist = [x for x in multipleseq if x.id == loc_full_allele]
        loc_full_protein = mslist[0]
        #loc_full_protein = multipleseq[id==loc_full_allele]
        full_protein = loc_full_protein.seq
        
        # align coordinate system based on reference sequence.
        # uses two field allele if full allele not present.
        # does not overwrite full allele if present.
        # necessary for earlier database versions.
        offset = self.coordinate(regexes[self.refseq_full[loc]], full_protein)
        self.hlaProteinOffset[loc] = offset

        return

    def adjust(self, multipleseq, loc):
        # then adjust end position based on dashes.
        new_end, new_end_inc = self.adjust_end(multipleseq, loc, self.ard_start_pos[loc],
                                            self.ard_start_pos_incomplete[loc],
                                            self.ard_end_pos[loc],
                                            self.ard_end_pos_incomplete[loc],
                                            prev=0, prev_inc=0)
        self.ard_end_pos[loc] = new_end
        self.ard_end_pos_incomplete[loc] = new_end_inc
        return


    def generate_IMGT(self, HLA_full_alseq):
        outfile = open("./IMGT_HLA_Full_Protein_" + str(self.dbversion) + ".txt", "w+")
        outfile.write("Allele\tFull_Protein\n")
        for allele_loctype in HLA_full_alseq:
            outfile.write("HLA-" + allele_loctype + "\t" +
                        str(HLA_full_alseq[allele_loctype]) + "\n")
        outfile.close()
        return

    def remove_gap(self):
        loci =  ['A', 'B', 'C', 'DRB1', 'DRB3', 'DRB4', 'DRB5', 'DQA1', 'DQB1', 'DPA1', 'DPB1']
        refgaps2 = {}
        refgaps4 = {}
        for locus in loci:
            seq2 = str(self.HLA_seq[self.refseq[locus]].seq)
            seq4 = str(self.HLA_full_allele[self.refseq_full[locus]].seq)
            refgaps2[locus] = [i for i, ltr in enumerate(seq2) if ltr == '-']
            refgaps4[locus] = [j for j, lttr in enumerate(seq4) if lttr == '-']
        for x2 in self.HLA_seq.keys():
            locus = x2.split('*')[0]
            gapseq2 = str(self.HLA_seq[x2].seq)
            seq2x = ''.join([gapseq2[i] for i in range(len(gapseq2)) if not(gapseq2[i] == '-' and (i in refgaps2[locus]))])
            self.HLA_seq[x2].seq = Seq(seq2x)
        for x4 in self.HLA_full_allele.keys():
            locus = x4.split('*')[0]
            gapseq4 = str(self.HLA_full_allele[x4].seq)
            seq4x = ''.join([gapseq4[i] for i in range(len(gapseq4)) if not(gapseq4[i] == '-' and (i in refgaps4[locus]))])
            self.HLA_full_allele[x4].seq = Seq(seq4x)
        return

    def main(self):
        loci =  ['A', 'B', 'C', 'DRB1', 'DRB345', 'DQA1', 'DQB1', 'DPA1', 'DPB1']
        self.HLA_full_allele = {} # Full four-field allele names
        #self.HLA_full_alseq = {} # Only used for generating IMGTHLA files
        self.HLA_seq = {} # Two-field
        regex = '\w*\*\d*\:\d*'
        suffixes = ["L", "S", "C", "A", "Q", "N"]

        for locus in loci:
            loc_full_alseq = {}
            seq_filename = "{}nuc_msf/{}_nuc_{}.msf".format(pathloc,locus,self.dbversion)
            seq_filename = Path(seq_filename)
            if path.exists(seq_filename) == False:
                print("Downloading requested MSF nucleotide files for locus " + locus + "...")
                url = "https://raw.githubusercontent.com/ANHIG/IMGTHLA/{}/msf/{}_nuc.msf".format(self.dbversion, locus)
                r = requests.get(url)
                with open(seq_filename, 'wb') as f:
                    f.write(r.content)
            else: 
                print("MSF nucleotide files already downloaded")
            multipleseq = AlignIO.read(seq_filename, format="msf")
            
            if locus == 'DRB345':
                for loc in ['DRB3', 'DRB4', 'DRB5']:
                    self.reference(multipleseq, loc)
                    self.adjust(multipleseq, loc)
            else: 
                self.reference(multipleseq, locus)
                self.adjust(multipleseq, locus)

            for record in multipleseq:
                loc_full_allele = record.id
                (loc,full_allele) = loc_full_allele.split("*")
                # append the suffix - needed for null alleles
                # index starts at 1 since some loci characters are also suffixes
                separator = loc_full_allele.find("*")
                if any(x in loc_full_allele[separator:] for x in suffixes):
                    loc_two_field_allele = re.match(regex, loc_full_allele).group() + \
                                        loc_full_allele[-1]
                else:
                    loc_two_field_allele = re.match(regex, loc_full_allele).group()
                full_protein = record.seq
                nogap = full_protein

                loc_full_alseq[loc_full_allele] = SeqRecord(nogap)

                # skip missing sequences in IMGT/HLA file
                if (len(full_protein) <10): # #NAME?
                    print ("Missing Sequence:" + loc_full_allele)
                    continue

                mature_protein = full_protein[self.getMatureProteinOffset(loc):]
                mrecord = SeqRecord(mature_protein)


                # full allele name
                self.HLA_full_allele[loc_full_allele] = mrecord

                # don't overwrite two-field alleles with new sequences - more likely
                # to be incomplete
                if (loc_two_field_allele not in self.HLA_seq):
                    self.HLA_seq[loc_two_field_allele] = mrecord

                # print (HLA_seqrecord_dict[allele])
                
                # TODO - add feature annotation to SeqIO object
                # https://biopython.org/wiki/SeqRecord
                # e.g. - which allele contain Bw4/Bw6 epitopes, bind LILRB1, etc
                # https://www.biostars.org/p/57549/

                # print (HLA_seqrecord_dict[allele].seq)
            
            #!GB!# Commented out these two lines as well as the generate_IMGT()
            # function call, since the process is intensive and does not need to be
            #!GB!# run every time the code is used. Might should consider a switch to
            # splitting a HLA_full_alseq DataFrame by locus and doing all of the work in
            #!GB!# a single function definition.
            #loc_full_alseq = remove_ins(loc_full_alseq)
            #HLA_full_alseq.update(loc_full_alseq)

        #generate_IMGT(HLA_full_alseq)

In [15]:
nuc_mm = NucMatch(dbversion=3400)

MSF nucleotide files already downloaded
MSF nucleotide files already downloaded
MSF nucleotide files already downloaded
MSF nucleotide files already downloaded
MSF nucleotide files already downloaded
MSF nucleotide files already downloaded
MSF nucleotide files already downloaded
MSF nucleotide files already downloaded
MSF nucleotide files already downloaded


In [76]:

class Impute():

    suffixes = ["L", "S", "C", "A", "Q", "N"]

    def __init__(self, suffixes=suffixes):
        self.suffixes = suffixes    
        self.main()
        


    def ungap(self, dataframe, refseq, loc):
        # The dashes will be put at the beginning of every set of possible
        # polymorphisms per residue.
        # This is to prevent all of the '-' characters from being sent to front
        i = 0
        j = 0
        new_cols = {}
        tNum = 0

        for column in dataframe:
            num = int(column[1:])
            if (column[0] == '-'):
                if dataframe.loc[refseq[loc]][column] == 1:
                    tNum = num
                    i += 1
                    j += 1
                    new_col = (str(num - i) + '_INS_' + str(j))
                    new_cols[column] = new_col
            else:
                if tNum == num:
                    new_col = (str(column[0]) + str(num - i) + '_INS_' + str(j))
                    new_cols[column] = new_col
                else:
                    new_col = (str(column[0]) + str(num - i))
                    new_cols[column] = new_col

        dataframe = dataframe.rename(columns=new_cols)
        return dataframe


    # returns list of indexes for dash characters in all sequences
    def findIns(self, sequence):
        seqIns = []
        idx = sequence.find('-')
        if (idx != -1):
            seqIns.append(idx)
        while (idx != -1):
            idx = sequence.find('-', idx + 1)
            if (idx != -1):
                seqIns.append(idx)
        return seqIns


    # removes indexes if also in refseq[loc]
    def checkComplete(self, sequence, seqIns):
        checkIns = []
        idx = sequence.find('-')
        if (idx != -1):
            checkIns.append(idx)
        while (idx != -1):
            idx = sequence.find('-', idx + 1)
            if (idx != -1):
                checkIns.append(idx)
        checkIns = [x for x in checkIns if x not in seqIns]
        return checkIns


    # translate the nucleotide CDS into the amino acid sequence for all alleles
    def translate_nuc(self, nuc_dict, seqIns):
        translated = {}
        incorrect = []
        print("Tranlsation in progress...")
        for record in tqdm(nuc_dict.keys()):
            nuc_seq = SeqRecord(Seq(nuc_dict[record]))
            try:
                aa_seq = SeqRecord(seq=nuc_seq.seq.translate(cds=False,
                                                            to_stop=True))
            except TranslationError:
                incorrect.append(record)
                new_seq = str(nuc_seq.seq)
                new_sequel = []
                new_sequel[:] = new_seq
                new_sequel = [new_sequel[x]
                            for x in range(0, len(new_sequel))
                            if (x not in seqIns)]
                nuc_seq = SeqRecord(Seq(''.join(new_sequel)))
                aa_seq = SeqRecord(seq=nuc_seq.seq.translate(cds=False,
                                                            to_stop=True))
            translated[record] = aa_seq
        return translated, incorrect

    # add back gaps that are present in the reference (and other) alleles so that
    # arrays are the same length
    def correction(self, incorrect, translated, aaIns):
        for each in incorrect:
            for insert in aaIns:
                translated[each] = translated[each][:insert] + '-' + \
                    translated[each][insert:]
        return translated

    # complete null alleles so that arrays are the same length
    def finish_null(self, refseq, repDict):
        suffixes = self.suffixes
        removal = []
        length = len(repDict[refseq])
        for entry in repDict.keys():
            separator = entry.find("*")
            if any(x in entry[separator:] for x in suffixes):
                if (entry[-1] == "N"):
                    trunk = len(repDict[entry])
                    diff = length - trunk
                    if (diff > 0):
                        filler = "*" * diff
                        fillist = list(filler)
                        fix = repDict[entry] + fillist
                        repDict[entry] = fix
                else:
                    removal.append(entry)
        for bad in removal:
            del repDict[bad]
        return repDict


    # Normalized Hamming distance is based on the number of sequence differences
    #   divided by the number of sequence positions that are defined in both
    #   sequences.
    def distmat(self, locDict, test=True):
        matDist = {}
        print("Generating distance matrix...")
        for lKey in tqdm(locDict.keys()):
            rDist = {}
            for locKey in locDict.keys():
                # needed for sequences with no overlaps in complete sequence
                i = 1
                diff = 0
                if test:
                    x = locDict[lKey]
                    y = locDict[locKey]

                    # compare sequences position-to-position
                    for j in range(0, len(x)):
                        if (x[j] != '-') and (y[j] != '-'):
                            if (x[j] == y[j]): 
                                i += 1
                            else:
                                i += 1
                                diff += 1
                    rDist[locKey] = diff/i
                else:
                    if (lKey != locKey):
                        x = locDict[lKey]
                        y = locDict[locKey]

                        # compare sequences position-to-position
                        for j in range(0, len(x)):
                            if (x[j] == '-') or (y[j] == '-'):
                                continue
                            else:
                                if (x[j] == y[j]): 
                                    i += 1
                                else:
                                    i += 1
                                    diff += 1
                        rDist[locKey] = diff/i
            matDist[lKey] = rDist
        disFrame = pd.DataFrame.from_dict(matDist)
        return disFrame

    # Recursive algorithm for nearest 10 neighbor voting for sequence inference.    
    def seqvote(self, rVal, rKey, disFrame, locDict, start):
        rVal = rVal - 1
        votes = {}
        topten = {}
        new_val = '-'
        for n in list(disFrame.columns[start:start+10]):
            topten[n] = disFrame.loc[rKey][n]

        for neighbor in topten.keys():
            n_acid = locDict[neighbor][rVal]
            if (n_acid not in votes.keys() and n_acid != "-"):
                votes[n_acid] = -abs(float(topten[neighbor]))
            elif (n_acid == "-"):
                next
            # for additional votes for a single position, add value
            # to increase the likelihood of selection
            else:
                votes[n_acid] += abs(float(topten[neighbor]))

            try:
                new_val = max(votes, key = lambda x: votes[x])
            except ValueError:
                start += 10
                return self.seqvote(rVal, rKey, disFrame, locDict, start)
        return new_val

    # nearest 10 vote based on distance matrix
    # TODO (gbiagini) - this function is not yet complete
    def nearest10(self, loc, locDict, disFrame, rPos):
        disFrame = disFrame.filter(regex=".*[^NQLSCA]$", axis=1)
        with open(pathloc + "data/nearest/{}_topten.txt".format(loc), "w+") as handle:
            handle.write("NEAREST 10 NEIGHBORS FOR EACH IMPUTED ALLELE + \n")
            handle.write("Format for neighbors is $ALLELE (HamDist for NA Seq)\n")
            print("Nearest 10 imputation...")
            for rKey in tqdm(rPos.keys()):
                disFrame = disFrame.sort_values(by=rKey, axis=1, ignore_index=False)
                # pull top 10 closest alleles
                handle.write("Imputed allele:\t\t\t\t" + rKey + "\n")
                topten = {}
                if str(disFrame.columns[0]) != rKey:
                    i = 0
                    for n in list(disFrame.columns[0:10]):
                        i += 1
                        topten[n] = disFrame.loc[rKey][n]
                        handle.write("Neighbor #" + str(i) + ":\t\t\t\t" + n + " (" +
                                    str(topten[n]) + ")\n")
                        start = 0
                else:
                    i = 0
                    for n in list(disFrame.columns[1:11]):
                        i += 1
                        topten[n] = disFrame.loc[rKey][n]
                        handle.write("Neighbor #" + str(i) + ":\t\t\t\t" + n + " (" +
                                    str(topten[n]) + ")\n")
                        start = 1
                # infers sequence from nearest 10 neighbor vote
                #!# if all 10 nearest neighbors have '-' at a position,
                # the search expands to other positions
                for rVal in rPos[rKey]:
                    new_val = self.seqvote(rVal, rKey, disFrame, locDict, start)

                    locDict[rKey] = locDict[rKey][:rVal] + \
                                new_val + locDict[rKey][rVal + 1:]
        return locDict

    def impute(self, loc, locDict, refseq, aaDict):
        seqIns = self.findIns(locDict[refseq])
        aaIns = self.findIns(str(aaDict[refseq].seq))
        #recompute = True
        if os.path.isfile("{}data/locdist/{}.csv".format(pathloc,loc)):
            recompute = False
        else:
            recompute = True
        if recompute:
            # TODO (gbiagini) - modify this to simply update previous distance
            #  matrix, rather than re-do the entire computation
            print("New alleles detected - must generate distance matrix!")
            replacePos = {}
            binDict = {}
            
            for key in locDict.keys():
                replacePos[key] = self.checkComplete(locDict[key], seqIns)

            disFrame = self.distmat(locDict)
            disFrame.to_csv(pathloc+"data/locdist/{}.csv".format(loc))
            rPos = {r: replacePos[r] for r in replacePos.keys() if
                    (len(replacePos[r]) != 0)}
            del(replacePos)
            
        else:
            replacePos = {}
            for key in locDict.keys():
                    replacePos[key] = self.checkComplete(locDict[key], seqIns)
            rPos = {r: replacePos[r] for r in replacePos.keys() if
                    (len(replacePos[r]) != 0)}
            del (replacePos)

            # TODO (gbiagini) - Need to fix this for situations in which the
            #  distance matrix is regenerated (make index_col = 0)

            # Modified to use complete distance matrix instead of previously used
            #   exclusionary distance matrix
            disFrame = pd.read_csv(pathloc+"data/locdist/{}.csv".format(loc), index_col=0)

        disFrame = disFrame.transpose()
        locDict = self.nearest10(loc, locDict, disFrame, rPos)
        locDict, incorrect = self.translate_nuc(locDict, seqIns)
        del(disFrame)
        translated = self.correction(incorrect, locDict, aaIns)
        return translated
        
    def main(self):
        aaDict = aa_mm.HLA_full_allele
        refseq = nuc_mm.refseq_full
        HLA_seq = nuc_mm.HLA_full_allele
        for loc in aa_mm.refseq:
            #if loc in ['A', 'B', 'C', 'DRB1', 'DRB3']:
                #continue
            print("Processing locus " + loc + "...")
            locDict = {newKey: str(HLA_seq[newKey].seq) for newKey in HLA_seq.keys()}
            newDict = {locKey: locDict[locKey] for locKey in locDict.keys() if (
                locKey.split('*')[0] == loc)}
            locDict = newDict
            del (newDict)
            imputed = self.impute(loc, locDict, refseq[loc], aaDict)
            # creates list from sequence strings for Pandas dataframe
            repDict = {repKey: list(imputed[repKey]) for repKey in imputed.keys()}
            del(imputed)
            repDict = self.finish_null(refseq[loc], repDict)
            impnew = {iKey: ''.join(repDict[iKey]) for iKey in repDict.keys()}
            my_records = []
            for pkey in impnew.keys():
                record = SeqRecord(
                    Seq(impnew[pkey]),
                    id=pkey,
                )
                my_records.append(record)
            SeqIO.write(my_records, "{}ifasta/{}_{}.fasta".format(pathloc,loc,nuc_mm.dbversion), "fasta")

            #repFrame = pd.DataFrame.from_dict(repDict)
            #repFrame = repFrame.transpose()
            # repFrame = pd.get_dummies(repFrame, prefix_sep='')
            # lambda function to rename columns with string character first then
            # position
            #repFrame = repFrame.rename(
            #   mapper=(lambda x: (str(x[-1]) + str(int(x[:-1]) + 1))), axis=1)
            #repFrame.index.names = ['allele']
            #repFrame = self.ungap(repFrame, refseq, loc)
            #repFrame.to_csv(pathloc+"imputed/{}_imputed.csv".format(loc), index=True)
            #print("Done with locus {}".format(loc))

In [ ]:
Impute()

Processing locus A...
New alleles detected - must generate distance matrix!
Generating distance matrix...
